# Annotate new data to improve Misogyny Detection models using Rubrix and biome.text

Hey there! In this guide, we will show you how we used Rubrix to annotate new data and use this new data to improve existing Deep Learning models. Our use case was Automatic Misogyny Detection (AMI): NLP models able to detect the underlying misogyny on a given text. Ground-breaking work is being made every year on this subject, with shared tasks and new models that push the performance of these models closer and closer to be implemented in apps, social networks, and other digital environments. 

Alongside Rubrix, we used our amazing NLP libary [biome.text](https://github.com/recognai/biome-text) to train models with a simple workflow. Rubrix is compatible with almost any library or service, so we were able to work back and forth with both of them. 

If it's your first time around, we recomend to start by reading our documentation [front page](https://docs.rubrix.ml/en/stable/). After that, you can continue by reading the Getting Started section. 

## Our objective

One of our team members, Ignacio, was preparing his Bachelor's thesis on the subject of Automatic Misogyny Detection. In this setting, we wanted to use the potential of Rubrix on our favor, going beyond the classic linear workflows that are so common in Deep Learning: gathering some data, preprocessing it, training a model and start making inference. Rubrix breaks that scheme, and allowed us to use a man-in-the-middle approach with retraining and fine-tuning and to add new data in follow-up trainings. 

We want to talk you more about it in this guide. It is not intended to be a tutorial, even though there will be snippets of code to reproduce our process and get to similar result (we will simplify some aspects of it to keep it light-weighted), but more of a story. We will focus on the process and on what we learnt along the way. 

## Background

The Bachelor's thesis of Ignacio was focused on the Spanish language, and datasets of misogyny in Spanish text is nothing but scarce. This was the first hardship: finding datasets to start training some models. Luckily, there is a very important community of shared tasks focused on the matter, covering a lot of non-English languages. We started working with data from [IberEval 2018](https://sites.google.com/view/ibereval-2…), a shared-task that offered a compilation of tweets, analyzed by experts and classified into 5 different misogyny categories. We started training our first model with around three thousand instances of annotated data. 

Afterwards, the [EXIST](http://nlp.uned.es/exist2021/) task at [IberLEF 2021](https://sites.google.com/view/iberlef2021) came to our sight. We saw there a perfect opportunity for our work on misogyny detection, for two reasons:

*   To test our approach on misogyny detection on a different subdomain, with different data.
*   To search for data to expand our three-thousand-or-so instances.

We covered our participation in the EXIST task in an [entry of our blogpost](https://medium.com/recognai/against-sexism-like-a-machine-2ae9227881ef). After we submitted our runs, we shifted our efforts towards integrating these new data into our corpus. The categorization was different: IberEval 2018 used a categorization system of 5 categories and 2 targets, but IberLEF 2021 didn't have a target field, and its categories were different. 

With the classic approach of Deep Learning, our journey would have ended there, with two different, incompatible datasets. But, thanks to Rubrix, it was only the beginning. 

## Annotation as a single agent

Our objective was to adapt data from EXIST shared task to the standard of IberLEF 2021. Therefore, as single annotators, we uploaded the dataset from IberLEF 2021, and begin to explore its predictions and annotations.

If you want to upload the training set of the EXIST task, follow the snippet below:

In [ ]:
annotation_df = pd.read_csv('https://raw.githubusercontent.com/ignacioct/Temis/main/datasets/IberLEF%202021/Spanish/EXIST2021_test_labeled_spanish.csv')

records = []

for index, row in annotation_df.iterrows():

    item = rb.TextClassificationRecord(
        id=index,
        inputs={
            "text": row["text"]
        },
        multi_label=True,
        metadata={
            "task1_annotation": row['task1'],
            "task2_annotation": row['task2'],
        }
    )

    records.append(item)

rb.log(records=records, name="single_annotation", tags={"project": "misogyny", "annotator": "ignacio"})

Once we've logged our annotation dataset into Rubrix, we can start annotating on the UI. Let's quickly remember how it's done

1. Open Rubrix in your browser. If you're running it locally, it is usually running on [http://localhost:6900](http://localhost:6900).
2. Select the `single_annotation` dataset.
3. On the upper-right corner, toggle the `Annotation mode`. 
4. Start selecting the categories that you think fit the input text. If you don't know Spanish, don't worry! 15 instances are not going to change the final model that much, and you will still learn how to annotate.
5. For each instance you can annotate a category by pressing it, discarding the record (if you think it does not fit the problem domain), or leave it without an annotation.

## Annotating as a team

We arraged a team of 5 different annotators, which worked over a week to transform instances from the EXIST standard to the one from IberEval 2018. For doing so, we needed a way to merge several annotations of the same instance into one, preserving the will of the majority, and that's when the Inter-Annotator Agreement (IAA) comes in handy. There many different types of IAAs, some based on rules and others based on statistics.

Here's a simplifaction of our IAA as a rule system:
* For an instance to be annotated with a category, there must be consensus of, at least, two annotators.
* If there's consensus in an sexism category, and other annotators find there's no sexism in the instance, it will be discarded.

Our team of annotators was formed by Amélie, Leire, Javier, Víctor and Ignacio. In the next cells, you can find a cell that logs the original annotations made by our annotators (the non-annotated version is the one downloaded in the previous section). After that, we will retrieve these annotated datasets from Rubrix using the `load` command.

If you want to explore all the datasets, code and resources used in the whole thesis, you can find them at [Temis Github page](https://github.com/ignacioct/Temis). Come to say hi!

In [6]:
annotation_1_df = pd.read_json('https://raw.githubusercontent.com/ignacioct/Temis/main/datasets/Annotation/temis_retraining_1.json')
annotation_2_df = pd.read_json('https://raw.githubusercontent.com/ignacioct/Temis/main/datasets/Annotation/temis_retraining_2.json')
annotation_3_df = pd.read_json('https://raw.githubusercontent.com/ignacioct/Temis/main/datasets/Annotation/temis_retraining_3.json')
annotation_4_df = pd.read_json('https://raw.githubusercontent.com/ignacioct/Temis/main/datasets/Annotation/temis_retraining_4.json')
annotation_5_df = pd.read_json('https://raw.githubusercontent.com/ignacioct/Temis/main/datasets/Annotation/temis_retraining_5.json')

Now, let's log this information into Rubrix. We are going to an approach similar to our single-agent annotation. We are showing you how to log one of the datasets, you just have to repeat the process and change the names of the logged datasets, so get logged separately, and each agent knows in which dataset she or he should annotate. 

In [2]:
import pandas as pd
import rubrix as rb

In [7]:
annotation_1_df.head()

,inputs,prediction,annotation,prediction_agent,annotation_agent,multi_label,explanation,metadata,status,event_timestamp
0,{'text': 'Y te golpeo más fuerte perra.'},"[[active, 0.9819192886], [discredit, 0.4090008...","[active, sexual_harassment]",temis_multilabel,@recognai,True,NaN,"{'task1_annotation': 'sexist', 'task2_annotati...",Validated,NaN
1,{'text': '@wthxtme @GOLDENMXM Castigado tres m...,"[[non-sexist, 1.0], [active, 0.0192600936], [p...",None,temis_multilabel,None,True,NaN,"{'task1_annotation': 'non-sexist', 'task2_anno...",Default,NaN
2,{'text': '@MaricaIndomito jajajaj el me he top...,"[[non-sexist, 1.0], [active, 0.016454557], [di...",None,temis_multilabel,None,True,NaN,"{'task1_annotation': 'sexist', 'task2_annotati...",Default,NaN
3,{'text': 'Resulta que hoy es el día de la juve...,"[[non-sexist, 1.0], [active, 0.019965389700000...",None,temis_multilabel,None,True,NaN,"{'task1_annotation': 'non-sexist', 'task2_anno...",Default,NaN
4,{'text': '@EstefMolina_ Los que dicen que cree...,"[[non-sexist, 1.0], [passive, 0.1393957436], [...",None,temis_multilabel,None,True,NaN,"{'task1_annotation': 'sexist', 'task2_annotati...",Default,NaN


In [ ]:
records = []

for index, row in annotation_1_df.iterrows():

    item = rb.TextClassificationRecord(
        id=index,
        inputs={
            "text": row["text"]
        },
        annotation=row["annotation"],
        annotation_agent="annotator 1",
        multi_label=True,
        metadata={
            "task1_annotation": row['task1'],
            "task2_annotation": row['task2'],
        }
    )

    records.append(item)

rb.log(records=records, name="annotation_misogyny_1", tags={"project": "misogyny", "annotator": "annotator 1"})

One thing that should be remember is that, for divulgative purpouses, we are simplyfing the complexity of the problem. You can find more information about how the labels in which our agents annotated are divided into two subcategories [here](https://github.com/ignacioct/Temis#predictions).

After our logging and exploration, we can go ahead and load these datasets from Rubrix.

In [ ]:
annotation_1 = rb.load("annotation_misogyny_1").set_index("id").sort_index()
annotation_2 = rb.load("annotation_misogyny_2").set_index("id").sort_index()
annotation_3 = rb.load("annotation_misogyny_3").set_index("id").sort_index()
annotation_4 = rb.load("annotation_misogyny_4").set_index("id").sort_index()
annotation_5 = rb.load("annotation_misogyny_5").set_index("id").sort_index()

`rb.load()` returns a Pandas Dataframe. We will use this library to merge our annotations into a single dataset.

In [ ]:
# We will use this tool to count ocurrences in list
from collections import Counter

annotation_final = pd.DataFrame(columns=['id','text', 'annotation', 'annotation_agent'])

# Iterating through the datasets, all of them has the same length
for i in range(len(annotation_anna)):
    
    # Extracting the annotated categories by each annotator
    category_annotated_anna = annotation_anna.iloc[i]["annotation"]
    category_annotated_bob = annotation_bob.iloc[i]["annotation"]
    category_annotated_celia = annotation_celia.iloc[i]["annotation"]
    
    # Merging the annotations into a list
    annotated_categories = [category_annotated_anna, category_annotated_bob, category_annotated_celia]
    # Flattening the list (if there is annotation, it is saved as an individual list)
    if not None in annotated_categories:
        annotated_categories = [item for sublist in annotated_categories for item in sublist] 
    
    # If all the elements in the list are None, we can return 'non-annotated'
    if all(annotation is None for annotation in annotated_categories):
        merged_annotation = 'non-annotated'    
    
    # Counting the annotations
    counted_annotations = Counter(annotated_categories)
    
    # Checking if the element with the most number of annotations follows the rules to be annotated
    if counted_annotations[max(counted_annotations, key=counted_annotations.get)] >= 2 and "0" not in counted_annotations:
        merged_annotation = max(counted_annotations, key=counted_annotations.get)
        
    else:
        merged_annotation = 'no-consensus'
        
        
    # As all elements in each row of the DataFrame except the annotations are the same, we can
    # retrieve information from any of the annotators. In our case is Anna.
    annotation_final = annotation_final.append({
        'id': annotation_anna.iloc[i]["metadata"]["id"],
        'text': annotation_anna.iloc[i]["inputs"]["text"],
        'annotation': merged_annotation,
        'annotation_agent': 'Anna, Bob and Celia',
    }, ignore_index=True)

## Appendix

Here are some procedures we've made for this guide that were kept on the background. If you want to reproduce all our steps, including the training of models and some extra parts, we will give provide with cells to do so! Feel free to change anything and try new stuff, and tell us if you have some doubts our find something cool at our [Github forum](https://github.com/recognai/rubrix/discussions)

### Dependencies & Installs

During this guide, we've provided some minimal code for our use case. However, to reproduce exactly our process, you will firstly need to install Rubrix, biome.text and pandas. We will also import them.

In [ ]:
%pip install -U git+https://github.com/recognai/biome-text
%pip install rubrix
%pip install pandas
exit(0)  # Force restart of the runtime

In [ ]:
from biome.text import *
import pandas as pd
import rubrix as rb

### Training our first model

To reproduce a simplified version of the first trained model, before annotation, you can execute the following cells. We've already searched for good-enough configurations, so you can skip that step.

Let's start by loading the datasets

In [ ]:
# Loading the datasets
training_ds = Dataset.from_csv('https://raw.githubusercontent.com/ignacioct/Temis/main/datasets/IberEval%202018/training_full_df.csv', index=False)
test_ds = Dataset.from_csv('https://raw.githubusercontent.com/ignacioct/Temis/main/datasets/IberEval%202018/test_df.csv', index=False)

Creating NLP pipelines with biome.text is quick and convenient! We performed an HPO process on the background, to find suitable hyperparameters for this domain, so let's use them to create our first AMI model. Note that we're making a pipeline with BETO, a Spanish Transformer model, at the head. To learn more about what a Transformer is, please visit the [Transformer guide of biome.text](https://recognai.github.io/biome-text/v3.0.0/documentation/tutorials/4-Using_Transformers_in_biome_text.html).

In [ ]:
pipeline_dict = {
    "name": "AMI_first_model",
    "features": {
        "transformers": {
            "model_name": "dccuchile/bert-base-spanish-wwm-cased", # BETO model
            "trainable": True,
            "max_length": 280,  # As we are working with data from Twitter, this is our max length
        }
    },
    "head": {
        "type": "TextClassification",
        
        # These are the possible misogyny categories.
        "labels": [
            'sexual_harassment',
             'dominance',
             'discredit',
             'stereotype',
             'derailing',
             'non-sexist'
        ],
        "pooler": {
            "type": "lstm",
            "num_layers": 1,
            "hidden_size": 256,
            "bidirectional": True,
        },
    },
}

pl = Pipeline.from_config(pipeline_dict)

In [ ]:
trainer_config = TrainerConfiguration(
    optimizer={
        "type": "adamw",
        "lr": 0.000023636840436059507,
        "weight_decay": 0.01438297700463013,
    },
    batch_size=8,
    max_epochs=10,
)

In [ ]:
trainer = Trainer(
    pipeline=pl,
    train_dataset=training_ds,
    valid_dataset=test_ds,
    trainer_config=trainer_config
)

In [ ]:
trainer.fit()

After `trainer.fit()` stops, the results of the training and the obtained model will be in the output folder.

We can make some predictions, and take a look at the performance of the model.

In [ ]:
pl.predict("Las mujeres no deberían tener derecho a voto")